<a href="https://colab.research.google.com/github/leokan92/class/blob/main/Aula1_regras_de_associacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apyori
!pip install pyfpgrowth

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5955 sha256=06cc8c2ea31d19e157d9d87638bf1bf042f248f4a60aa662eb98e36eac16fa23
  Stored in directory: /root/.cache/pip/wheels/c4/1a/79/20f55c470a50bb3702a8cb7c94d8ada15573538c7f4baebe2d
Successfully built apyori
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5488 sha256=98f3816dc915a1c17afc77583e186ba8f402b21939720827497a74a2c9df7284
  Stored in directory: /root/.cache/pip/wheels/09/fc/dc/afff211038bfc745722d8d7e846e854e5791968b22c570a530
Successfully built pyfpgrowth


# Aula 1 - Regras de associação

##Definimos os dados usados na demonstração

In [ ]:
Transactions = [
    ['I1', 'I2', 'I5'],
    ['I2', 'I4'],
    ['I2', 'I3'],
    ['I1', 'I2', 'I4'],
    ['I1', 'I3'],
    ['I2', 'I3'],
    ['I1', 'I3'],
    ['I1', 'I2', 'I3', 'I5'],
    ['I1', 'I2', 'I3']
]

In [ ]:
order = ['I' + str(i) for i in range (1,6)]
display(order)

['I1', 'I2', 'I3', 'I4', 'I5']

### Definição dos min_supp e min_conf para filtrarmos as regras

In [ ]:
C = {}
L = {}
itemset_size = 1
min_support = 2/9
min_conf = 0.3
num_trans = len(Transactions)
Discarded = {itemset_size: []}
C.update({itemset_size : [[f] for f in order]})
display(C)

{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}

## Definimos as funções usadas pelo algoritmo

In [ ]:
def count_occurences(itemset, Transactions):
  # Inicializa a contagem de ocorrências como 0
  count = 0
  # Itera sobre todas as transações
  for i in range(len(Transactions)):
    # Verifica se o conjunto de itens (itemset) é um subconjunto da transação atual
    # Converte ambos itemset e transação em conjuntos para usar issubset()
    if set(itemset).issubset(set(Transactions[i])):
      # Se itemset é um subconjunto da transação, incrementa a contagem
      count += 1
  # Após iterar por todas as transações, retorna o número total de ocorrências do itemset
  return count


In [ ]:
def print_table(T, supp_count):
  print("Itemset | Frequency")
  for k in range(len(T)):
    print("{} : {}".format(T[k],supp_count[k]))
  print("\n\n")

In [ ]:
def join_two_itemsets(it1, it2, order):
  # Ex: [I1,I2] V [I1,I3] = [I1,I2,I3]
  # Ordena os dois conjuntos de itens (it1 e it2) de acordo com uma ordem predefinida
  it1.sort(key=lambda x: order.index(x))
  it2.sort(key=lambda x: order.index(x))

  # Compara os itens, exceto o último, de ambos os conjuntos
  for i in range(len(it1)-1):
    if it1[i] != it2[i]:
        # Se os itens iniciais são diferentes, retorna uma lista vazia (não podem ser unidos)
        return []

  # Verifica se o último item de it1 vem antes do último item de it2 na ordem predefinida
  if order.index(it1[-1]) < order.index(it2[-1]):
    # Retorna a união dos conjuntos, que é todos os itens de it1 mais o último item de it2
    return it1 + [it2[-1]]

  # Se as condições acima não são satisfeitas, retorna uma lista vazia
  return []

In [ ]:
def join_set_itemsets(set_of_its, order):
  # L = [[I1],[I2],[I3]]; L V L = [[I1,I2], [II,I3]...]
  # Inicializa uma nova lista para armazenar os conjuntos de itens unidos
  C = []
  # Itera sobre todos os pares possíveis dentro do conjunto fornecido
  for i in range(len(set_of_its)):
    for j in range(i+1, len(set_of_its)):
      # Tenta unir os pares de conjuntos de itens
      it_out = join_two_itemsets(set_of_its[i], set_of_its[j], order)
      # Se a união é bem-sucedida e resulta em um conjunto não vazio
      if len(it_out) > 0:
        # Adiciona o novo conjunto à lista C
        C.append(it_out)
  # Retorna a lista C com todos os novos conjuntos de itens unidos
  return C

In [ ]:
def get_frequent(itemsets, Transactions, min_support, prev_discarded):
  # Inicializa as listas para armazenar os conjuntos de itens frequentes, sua contagem de suporte e os novos descartados
  L = []
  supp_count = []
  new_discarded = []
  # Calcula o número total de transações
  num_trans = len(Transactions)

  # Obtém o tamanho atual da hierarquia dos conjuntos de itens descartados
  k = len(prev_discarded.keys())

  # Itera sobre cada conjunto de itens candidatos
  for s in range(len(itemsets)):
    # Inicializa a flag para verificar se o itemset foi descartado em iterações anteriores
    discarded_before = False
    # Se já houver conjuntos de itens descartados em níveis anteriores
    if k > 0:
      # Verifica se algum dos conjuntos descartados anteriormente é subconjunto do conjunto atual
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemsets[s])):
          discarded_before = True
          break
    # Se o conjunto de itens não foi descartado anteriormente
    if not discarded_before:
      # Conta as ocorrências do conjunto de itens atual nas transações
      count = count_occurences(itemsets[s], Transactions)
      # Se a contagem de suporte atender ao suporte mínimo
      if count/num_trans >= min_support:
        # Adiciona o conjunto de itens e sua contagem à lista de frequentes
        L.append(itemsets[s])
        supp_count.append(count)
      else:
        # Caso contrário, adiciona o conjunto de itens à lista de descartados
        new_discarded.append(itemsets[s])

  # Retorna os conjuntos de itens frequentes, sua contagem de suporte e a nova lista de descartados
  return L, supp_count, new_discarded

### Teste da função para atualizar C e L

In [ ]:
supp_count_L = {}
f, sup, new_discarded = get_frequent(C[itemset_size],Transactions,min_support, Discarded)
Discarded.update({itemset_size: new_discarded})
L.update({itemset_size: f})
supp_count_L.update({itemset_size: sup})

In [ ]:
print("L1")
print_table(L[1], supp_count_L[1])

L1
Itemset | Frequency
['I1'] : 6
['I2'] : 7
['I3'] : 6
['I4'] : 2
['I5'] : 2





## Execução do algoritmo para encontrar regras de associação

In [ ]:
# Inicializando o tamanho do conjunto de itens como o tamanho atual + 1
k = itemset_size + 1
# Inicializando a variável de convergência como False
convergence = False
# Iniciando um loop que continuará até que a convergência seja alcançada
while not convergence:
  # Atualiza o dicionário C com a união de conjuntos de itens de tamanho k-1
  C.update({k: join_set_itemsets(L[k-1], order)})

  # Imprime a tabela C de tamanho k
  print("Tabela C{}: \n".format(k))
  print_table(C[k], [count_occurences(it, Transactions) for it in C[k]])

  # Calcula os conjuntos de itens frequentes, suporte e itens descartados para o tamanho atual k
  f, sup, new_discarded = get_frequent(C[k], Transactions, min_support, Discarded)
  # Atualiza o dicionário de itens descartados
  Discarded.update({k: new_discarded})
  # Atualiza o dicionário L com os conjuntos de itens frequentes de tamanho k
  L.update({k: f})
  # Atualiza a contagem de suporte para os conjuntos de itens frequentes
  supp_count_L.update({k: sup})
  # Verifica se não há mais conjuntos de itens frequentes de tamanho k, o que indica convergência
  if len(L[k]) == 0:
    convergence = True
  else:
    # Caso ainda haja conjuntos de itens frequentes, imprime a tabela L de tamanho k
    print("Tabela L{}: \n".format(k))
    print_table(L[k], supp_count_L[k])
  # Incrementa o tamanho do conjunto de itens
  k += 1


Tabela C2: 

Itemset | Frequency
['I1', 'I2'] : 4
['I1', 'I3'] : 4
['I1', 'I4'] : 1
['I1', 'I5'] : 2
['I2', 'I3'] : 4
['I2', 'I4'] : 2
['I2', 'I5'] : 2
['I3', 'I4'] : 0
['I3', 'I5'] : 1
['I4', 'I5'] : 0



Tabela L2: 

Itemset | Frequency
['I1', 'I2'] : 4
['I1', 'I3'] : 4
['I1', 'I5'] : 2
['I2', 'I3'] : 4
['I2', 'I4'] : 2
['I2', 'I5'] : 2



Tabela C3: 

Itemset | Frequency
['I1', 'I2', 'I3'] : 2
['I1', 'I2', 'I5'] : 2
['I1', 'I3', 'I5'] : 1
['I2', 'I3', 'I4'] : 0
['I2', 'I3', 'I5'] : 1
['I2', 'I4', 'I5'] : 0



Tabela L3: 

Itemset | Frequency
['I1', 'I2', 'I3'] : 2
['I1', 'I2', 'I5'] : 2



Tabela C4: 

Itemset | Frequency
['I1', 'I2', 'I3', 'I5'] : 1





In [ ]:
from itertools import combinations, chain

def powerset(s):
  return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s)+1)))

In [ ]:
def write_rules(X, X_S, S, conf, supp, lift, num_trans):
  out_rules = ""
  out_rules += "Freq. Itemset: {}\n".format(X)
  out_rules += "   Rule: {} -> {} \n".format(list(S), list(X_S))
  out_rules += "   Conf: {0:2.3f} ".format(conf)
  out_rules += "   Supp: {0:2.3f} ".format(supp/ num_trans)
  out_rules += "   Lift: {0:2.3f} ".format(lift)
  return out_rules

In [ ]:
assoc_rules_str = ""
# Itera sobre todos os níveis de conjuntos de itens frequentes, exceto o nível 0
for i in range(1, len(L)):
  for j in range(len(L[i])):
    # Gera todos os subconjuntos possíveis do conjunto de itens frequentes
    s = list(powerset(L[i][j]))
    # Remove o próprio conjunto para evitar regras inválidas
    s.pop()
    for z in s:
      S = set(z)
      X = set(L[i][j])
      X_S = set(X-S)
      # Calcula o suporte do conjunto X
      sup_x = count_occurences(X, Transactions)
      # Calcula o suporte do conjunto X-S
      sup_x_s = count_occurences(X_S, Transactions)
      # Calcula a confiança da regra
      conf = sup_x/count_occurences(S, Transactions)
      # Calcula o lift da regra
      lift = conf/(sup_x_s / num_trans)
      # Verifica se a regra atende os critérios mínimos de confiança e suporte
      if conf >= min_conf and sup_x >= min_support:
        # Escreve a regra e adiciona à string de regras de associação
        assoc_rules_str += write_rules(X, X_S, S, conf, sup_x, lift, num_trans)


In [ ]:
print(assoc_rules_str)

Freq. Itemset: {'I1', 'I2'}
   Rule: ['I1'] -> ['I2'] 
   Conf: 0.667    Supp: 0.444    Lift: 0.857 Freq. Itemset: {'I1', 'I2'}
   Rule: ['I2'] -> ['I1'] 
   Conf: 0.571    Supp: 0.444    Lift: 0.857 Freq. Itemset: {'I1', 'I3'}
   Rule: ['I1'] -> ['I3'] 
   Conf: 0.667    Supp: 0.444    Lift: 1.000 Freq. Itemset: {'I1', 'I3'}
   Rule: ['I3'] -> ['I1'] 
   Conf: 0.667    Supp: 0.444    Lift: 1.000 Freq. Itemset: {'I1', 'I5'}
   Rule: ['I1'] -> ['I5'] 
   Conf: 0.333    Supp: 0.222    Lift: 1.500 Freq. Itemset: {'I1', 'I5'}
   Rule: ['I5'] -> ['I1'] 
   Conf: 1.000    Supp: 0.222    Lift: 1.500 Freq. Itemset: {'I2', 'I3'}
   Rule: ['I2'] -> ['I3'] 
   Conf: 0.571    Supp: 0.444    Lift: 0.857 Freq. Itemset: {'I2', 'I3'}
   Rule: ['I3'] -> ['I2'] 
   Conf: 0.667    Supp: 0.444    Lift: 0.857 Freq. Itemset: {'I2', 'I4'}
   Rule: ['I4'] -> ['I2'] 
   Conf: 1.000    Supp: 0.222    Lift: 1.286 Freq. Itemset: {'I2', 'I5'}
   Rule: ['I5'] -> ['I2'] 
   Conf: 1.000    Supp: 0.222    Lift: 1.286 

## Using Libs to find the association rule:

###Apriori

**Abordagem:**com base no princípio de geração de candidatos, onde começa identificando os itens individuais que atendem a um limite mínimo de suporte. Em seguida, ele estende esses conjuntos de itens, adicionando um item por vez, e verifica se os conjuntos de itens resultantes atendem ao limite de suporte.

**Eficiência:** A priori pode ser bastante lento e ineficiente para grandes conjuntos de dados, pois precisa verificar o banco de dados várias vezes para cada nível de conjuntos de itens que gera.

**Caso de uso:** Mais adequado para conjuntos de dados menores.

In [ ]:
from apyori import apriori

rules = apriori(transactions = Transactions,
                min_support = 0.003,
                min_confidence = 0,
                min_lift = 0,
                min_length = 2,
                max_length = 2)

results = list(rules)
for rule in results:
  print(rule)

RelationRecord(items=frozenset({'I1'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I1'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'I2'}), support=0.7777777777777778, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I2'}), confidence=0.7777777777777778, lift=1.0)])
RelationRecord(items=frozenset({'I3'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I3'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'I4'}), support=0.2222222222222222, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I4'}), confidence=0.2222222222222222, lift=1.0)])
RelationRecord(items=frozenset({'I5'}), support=0.2222222222222222, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I5'}), confidence=0.2222222222222222, lift=1.

###ECLAT (clustering de classes de equivalência e travessia de rede bottom-up):

**Abordagem:** ECLAT melhora o Apriori usando uma estratégia de busca em profundidade. Ele gera conjuntos de itens semelhantes ao Apriori, mas usa um formato de banco de dados vertical (listas de ID de transação para cada item). A interseção dessas listas de ID ajuda a contar rapidamente o suporte dos conjuntos de itens.

**Eficiência:** ECLAT é normalmente mais rápido que Apriori, pois reduz a necessidade de varrer todo o banco de dados repetidamente e tem melhor eficiência computacional devido à operação de interseção.

**Caso de uso:** eficaz para conjuntos de dados de tamanho médio e conjuntos de dados relativamente densos.

In [ ]:
rules = apriori(transactions = Transactions,
                min_support = 0.003,
                min_length = 2,
                max_length = 0)

results = list(rules)
for rule in results:
  print(rule)

RelationRecord(items=frozenset({'I1'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I1'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'I2'}), support=0.7777777777777778, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I2'}), confidence=0.7777777777777778, lift=1.0)])
RelationRecord(items=frozenset({'I3'}), support=0.6666666666666666, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I3'}), confidence=0.6666666666666666, lift=1.0)])
RelationRecord(items=frozenset({'I4'}), support=0.2222222222222222, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I4'}), confidence=0.2222222222222222, lift=1.0)])
RelationRecord(items=frozenset({'I5'}), support=0.2222222222222222, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'I5'}), confidence=0.2222222222222222, lift=1.

###Crescimento FP (crescimento de padrão frequente):

**Abordagem:** FP-Growth representa o banco de dados na forma de uma estrutura de dados compacta chamada FP-Tree (Frequent Pattern Tree) e então extrai a árvore construindo FP-Trees condicionais, cada uma associada a um item frequente.

**Eficiência:** É consideravelmente mais eficiente que Apriori e ECLAT, especialmente para grandes conjuntos de dados. Essa eficiência vem do fato de que ele só precisa passar pelo banco de dados duas vezes e não precisa gerar explicitamente conjuntos de candidatos.

**Caso de uso:** Altamente adequado para conjuntos de dados muito grandes.

In [ ]:
import pyfpgrowth
patterns = pyfpgrowth.find_frequent_patterns(Transactions, 2)
rules = pyfpgrowth.generate_association_rules(patterns, 0.3) # <- minimum probability threshold

for rule in rules:
  print(rule)

('I1',)
('I5',)
('I1', 'I2')
('I1', 'I5')
('I2', 'I5')
('I4',)
('I2',)
('I1', 'I3')
('I2', 'I3')
